In [2]:
import os, sys
import glob
import numpy as np
sys.path.append(os.path.join(os.getenv('LAMBO_HOME'), 'lambo/scripts/'))

from hsc_gaap.gaap import findReducedPatches


Overriding default configuration file with /projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/share/eups/Linux64/dustmaps_cachedata/g41a3ec361e+62d12e78cb/config/.dustmapsrc


In [3]:
def checkRun(tract, repo = '/scratch/gpfs/am2907/Merian/gaap/', output=True):
    if output:
        print(f'TRACT: {tract}')
    problem_patches=[]

    patches = findReducedPatches(tract)
    dir = os.path.join(repo, f"log/{tract}/")
    logs = np.array(glob.glob(dir + "*.o"))
    logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
    logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]

    if len(logs) != len(patches):
        print("MISSING PATCHES")

    error1 = "Wrote GAaP table to "
    error2 = "Running GAaP only on filters"
    error3 = "KeyError: 'g_ap03Flux'"
    for i, log in enumerate(logs):
        logfile = open(log, "r").read()
        
        if error1 not in logfile:
            if output:
                print (f'PROBLEM IN PATCH {logs_patches[i]}: Catalog not saved')
            problem_patches.append(logs_patches[i])

        if (error2 in logfile):
            if output:
                print (f'PROBLEM IN PATCH {logs_patches[i]}: Missing some filter images')
            problem_patches.append(logs_patches[i])
        
        elif (logfile.count("Finished the GAaP measureTask for band") < 5):
            if output:
                print (f'PROBLEM IN PATCH {logs_patches[i]}: Failed for {5 - logfile.count("Finished the GAaP measureTask for band")} bands')
            problem_patches.append(logs_patches[i])

        if error3 in logfile:
            if output:
                print (f'PROBLEM IN PATCH {logs_patches[i]}: Blend matching failed')
            problem_patches.append(logs_patches[i])

    if (len(problem_patches)==0) & (len(logs) == len(patches)):
        print ("NO PROBLEMS")
    print()

    return np.unique(problem_patches)

In [23]:
problems = checkRun(9074)

TRACT: 9074
PROBLEM IN PATCH 58: Failed for 3 bands
PROBLEM IN PATCH 63: Failed for 1 bands
PROBLEM IN PATCH 64: Failed for 2 bands
PROBLEM IN PATCH 67: Failed for 2 bands
PROBLEM IN PATCH 68: Failed for 1 bands
PROBLEM IN PATCH 71: Failed for 2 bands
PROBLEM IN PATCH 72: Failed for 1 bands
PROBLEM IN PATCH 78: Failed for 1 bands



In [24]:
len(problems)

8

In [25]:
print(", ".join(np.array(problems).astype(str)))

58, 63, 64, 67, 68, 71, 72, 78


---

# Fixing blending matching

In [101]:
%load_ext autoreload
%autoreload 2

In [102]:
from hsc_gaap.run_gaap import matchBlendedness

yay


In [7]:
blenderror ='Traceback (most recent call last):\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 198, in <module>\n    fire.Fire(runGaapRowColumn)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 141, in Fire\n    component_trace = _Fire(component, args, parsed_flag_args, context, name)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 475, in _Fire\n    component, remaining_args = _CallAndUpdateTrace(\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/fire/core.py", line 691, in _CallAndUpdateTrace\n    component = fn(*varargs, **kwargs)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 133, in runGaapRowColumn\n    matchBlendedness(tract, patch_cols, patch_rows, repo=repo)\n  File "/scratch/gpfs/am2907/Merian/gaap/lambo/scripts/hsc_gaap/run_gaap.py", line 187, in matchBlendedness\n    abs(gaapCat["g_ap03Flux"] - blendCat["g_apertureflux10_S20A"][ind]) < 10)\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/table/table.py", line 1872, in __getitem__\n    return self.columns[item]\n  File "/projects/HSC/LSST/stack_20220527/conda/envs/lsst-scipipe-4.1.0/lib/python3.10/site-packages/astropy/table/table.py", line 246, in __getitem__\n    return OrderedDict.__getitem__(self, item)\nKeyError: \'g_ap03Flux\''
repo = "/scratch/gpfs/am2907/Merian/gaap/"

In [8]:
def fixBlendMatching(tract, repo = '/scratch/gpfs/am2907/Merian/gaap/', blenderror=blenderror):
    
    # Find patches that didn't match
    print(f'TRACT: {tract}')
    problem_patches=[]

    patches = findReducedPatches(tract)
    dir = os.path.join(repo, f"log/{tract}/")
    logs = np.array(glob.glob(dir + "*.o"))
    logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
    logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]
    
    if len(logs) != len(patches):
        print("MISSING PATCHES")

    error3 = "KeyError: 'g_ap03Flux'"
    for i, log in enumerate(logs):
        logfile = open(log, "r").read()

        if error3 in logfile:
            problem_patches.append(logs_patches[i])

    for patch in problem_patches:
        print(f'PATCH: {patch}')
        log = logs[logs_patches==patch][0]
        logfile = open(log, "r").read()
        message = matchBlendedness(tract, [patch%9], [patch//9], repo=repo, matchdist=0.5, matchmag=10)
        newlog_cntnt = logfile.split(blenderror)[0] + f"\n {message} \n" + logfile.split(blenderror)[1]
        newlog = open(log, "w")
        newlog.write(newlog_cntnt)
        newlog.close()

In [129]:
tract=8526

In [13]:
checkRun(9010)

TRACT: 9010
PROBLEM IN PATCH 0: Catalog not saved
PROBLEM IN PATCH 0: Failed for 5 bands
PROBLEM IN PATCH 1: Failed for 3 bands
PROBLEM IN PATCH 2: Failed for 2 bands



array([0, 1, 2])

In [10]:
fixBlendMatching(8769)

TRACT: 8769
